In [2]:
import os
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
import krwordrank
import pandas as pd

current_dir = os.getcwd()

In [17]:
review_df = pd.read_csv(current_dir + '/data/review_movie_id.csv')

In [18]:
review_df.head()

,movie_id,review
0,38,시사회에서 보고왔습니다동화와 재즈뮤지컬의 만남 지루하지않고 재밌습니다
1,38,사랑과 꿈 그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다 배우들 연기는 두...
2,38,지금껏 영화 평가 해본 적이 없는데 진짜 최고네요 색감 스토리 음악 연기 모두 최고입니다
3,38,방금 시사회 보고 왔어요 배우들 매력이 눈을 뗄 수가 없게 만드네요 한편의 그림 같...
4,38,초반부터 끝까지 재미있게 잘보다가 결말에서 고국마 왕창먹음 힐링 받는 느낌들다가 막...


In [19]:
review_df.shape

(35420, 2)

In [20]:
new_review = "너무재밌어요 돈하나도안아까움ㅋㅋ"
movie_id = 38 # 스파이더맨?

In [21]:
new_review = normalize(new_review, english=True, number=False)

In [22]:
new_review

'너무 재밌어요 돈 하나도 안 아까움'

In [23]:
if len(new_review) > 0 :
    data = {'movie_id': movie_id, 'review': new_review}
    review_df = review_df.append(data, ignore_index=True)

In [24]:
review_df.tail()

,movie_id,review
35416,356,무간도를 능가하지는 못했지만 무간도급 재미의 영화 일단 시나리오가 찰지다 뭔가 쫀득...
35417,356,재미있을것 같네요
35418,356,부당거래의 후속작을 베를린이라고 생각했으나 바로 이 영화였다 박훈정의 신세계무간도의...
35419,356,첫 평점을 제가 남기는군요 영광입니다 사실 화요일에 시사회에서 봤었거든요 오랜만에 ...
35420,38,너무 재밌어요 돈 하나도 안 아까움


In [40]:
reviews = list(review_df[review_df['movie_id']==movie_id].review)

In [26]:
def define_stopwords(path):
    SW = set()

    with open(path, encoding="utf-8") as f:
        for word in f:
            SW.add(word.strip())
            
    # SW.add("있어요")
    
    return SW

In [27]:
def get_from_list(l, i, default=('', 0)):
    if len(l) <= i:
        return default
    else:
        return l[i]

In [38]:
def keyword_extraction(reviews):
    wordrank_extractor = KRWordRank(
        min_count = 7, # 단어의 최소 출현 빈도수 (그래프 생성 시)
        max_length = 5, # 단어의 최대 길이
        verbose = True
        )

    beta = 0.85    # PageRank의 decaying factor beta
    max_iter = 10

    try: 
        keywords, rank, graph = wordrank_extractor.extract(reviews, beta, max_iter)
        top_keywords = sorted(keywords.items(),
                key=lambda x:x[1],
                reverse=True)[:50]

        SW = define_stopwords(current_dir + "/data/stopwords-ko.txt")
        selected_top_keywords = []
        for word, r in top_keywords:
            
            if word in SW:
                continue
            selected_top_keywords.append((word, r))

        # 키워드 5개 추출
        return selected_top_keywords
            
    except:
        print("키워드 추출할 충분한 리뷰 데이터가 없습니다.")

In [41]:
keyword_extraction(reviews)

scan vocabs ... 
num vocabs = 9846
done = 10 Early stopped.


[('음악', 30.594262658688717), ('뮤지컬', 17.216111561463077), ('아름', 15.786654961015012), ('영상', 15.611933408234952), ('사랑', 15.476812862772855), ('꿈을', 15.377050917293797), ('여운이', 14.884349585387731), ('노래', 14.07101543240435), ('인생', 11.725790611962108), ('현실', 11.649690930377101), ('좋은', 11.60777789977812), ('감동', 10.196846460022364), ('좋고', 8.749296664775096), ('계속', 8.473742795671983), ('연기', 8.20311013819215), ('결말', 7.961094997590159), ('장면', 7.921166512841176), ('하는', 7.865032518043676), ('라이언', 6.92454979538581), ('남는', 6.890103452975571), ('연출', 6.639012297340128), ('재즈', 6.608836866402526), ('모든', 6.478891574141345)]

## 학습 테스트

### venom reviews

In [15]:
reviews = ['It’s a solid movie. Felt like it really did not have a big bad villain. Riot felt underused. Still a funny movie but not as good as it’s sequel',
'It was entertaining, but it lacked a true villain, the riot symbiote only bonded to Carlton Drake in the last act of the film. I did enjoy the banter between Eddie and Venom but the film is just basic. Still I enjoyed some bits, just not the film as a whole', 
'Brilliant stand alone venom movie. Although we are all used to venom being the villain, it was good to see the charater development of the of Eddie and the symbiote. Brilliant acting and character choice really brings the movie together.', 
'Fun movie but boring at times', 
'So funny I shat myself', 
'So stupid and fun and just totally Marvel all in one. Five Stars because it keeps my son entertained for hours.',
"This movie is amazing. I don't know why so many bad rewiews.",
"Absolute insane fun. Don't come in expected a cinematic masterpiece or a typical superhero movie. Come for the hilarious and insane dynamic between Eddie and Venom. This is not artistic or highly nuisanced film. This is a fun, hysterical romp with a human disaster and his equally disastrous parasite.",
"The characters/character of Eddie and Venom are what makes it good",
"Absolutely spectacular performance by Tom Hardy. This movie is what Deadpool thinks it is.",
"I'm not a fan of Tom Hardy, I think it's mostly his voice. However, I was pleasantly surprised at how well this movie was done. This was a great a job of introducing the character.",
"Great movie! Ready for the next instalment",
"Great action and comedy. Kind of a cheesy story at times but Venom's character makes up for it. Great portrayal by Tom.",
"The introduction to Venom in this first movie kind of feels off as well as the pacing but nonetheless great character development at the end of the movie makes it in my opinion better than the sequel",
"Venom shows us a diferent and more creatve origin for on my favorite characher of Marvel Comics, Tom Hardy made an awesome job as Eddie Brock with having funny cupel with Venom.",
"The movie is great. The origin feels a bit rushed and you dont learn much about venoms powers, and the villains motive is a bit strange, but the action scenes are brilliant and Tom Hardy is a fantastic Eddie Brock",
"The only redeeming qualities here are the look of venom, the voice, and the interactions between Eddie and Venom. Pretty much everything else is crap. Even if we ignore the fact that the movie just completely misses the point of Venom, it's just not a good story. Not a good villain, not a good origin, the character motivations are weak, and the final conflict isn't even that good. Also, Michelle Williams' character was completely pointless. Could have just made Dan Eddie's doctor, and the story would have been exactly the same."
]

In [16]:
len(reviews)

17

In [17]:
from googletrans import Translator
translator = Translator()

trans_review = []
for review in reviews:
    trans_review.append(translator.translate(review, dest='ko').text)
    

In [19]:
trans_review = [normalize(text, english=False, number=False) for text in trans_review]

In [26]:
keyword_extraction(trans_review)

scan vocabs ... 
num vocabs = 75
done = 10
['좋은', '재미있', '완전히', '캐릭터']


## 키워드 추출

In [3]:
df = pd.read_csv(current_dir+'/data/naver_movie_reviews.csv')
df.head()

,id,title,type,original_title,idx,movie_idx,score,text
0,0,모가디슈,Movie,NaN,192150.0,192150.0,1.0,정만식이 페미 옹호한다고 남자들한테 욕 쓴게 자꾸 생각나네... 만식아... 페미 ...
1,0,모가디슈,Movie,NaN,192150.0,192150.0,9.0,역사적인 내용과 스토리 괜찮았음 볼만함
2,0,모가디슈,Movie,NaN,192150.0,192150.0,8.0,신파없는 영화라서 너무 좋았어요 연기력과 연출력 만으로 이렇게 재밋게 만들수 있는데...
3,0,모가디슈,Movie,NaN,192150.0,192150.0,10.0,정말 잘짜여진 영화같습니다지루할틈 없이 잘봤습니다!!
4,0,모가디슈,Movie,NaN,192150.0,192150.0,1.0,페미들이 평점 조작한다는 화력지원으로 점수 올린다고해서 정상점수 만들기 운동에서 와슴


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112841 entries, 0 to 112840
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              112841 non-null  int64  
 1   title           112841 non-null  object 
 2   type            112841 non-null  object 
 3   original_title  83489 non-null   object 
 4   idx             112821 non-null  float64
 5   movie_idx       112773 non-null  float64
 6   score           112773 non-null  float64
 7   text            110399 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 6.9+ MB


In [9]:
df[df['title']=='모가디슈'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1000 non-null   int64  
 1   title           1000 non-null   object 
 2   type            1000 non-null   object 
 3   original_title  0 non-null      object 
 4   idx             1000 non-null   float64
 5   movie_idx       1000 non-null   float64
 6   score           1000 non-null   float64
 7   text            992 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 70.3+ KB


In [13]:
drop_idx = df[df['text'].isna()].index
df.drop(drop_idx, axis=0, inplace=True)

In [14]:
df.reset_index(drop=True, inplace=True)

In [15]:
df

,id,title,type,original_title,idx,movie_idx,score,text
0,0,모가디슈,Movie,NaN,192150.0,192150.0,1.0,정만식이 페미 옹호한다고 남자들한테 욕 쓴게 자꾸 생각나네... 만식아... 페미 ...
1,0,모가디슈,Movie,NaN,192150.0,192150.0,9.0,역사적인 내용과 스토리 괜찮았음 볼만함
2,0,모가디슈,Movie,NaN,192150.0,192150.0,8.0,신파없는 영화라서 너무 좋았어요 연기력과 연출력 만으로 이렇게 재밋게 만들수 있는데...
3,0,모가디슈,Movie,NaN,192150.0,192150.0,10.0,정말 잘짜여진 영화같습니다지루할틈 없이 잘봤습니다!!
4,0,모가디슈,Movie,NaN,192150.0,192150.0,1.0,페미들이 평점 조작한다는 화력지원으로 점수 올린다고해서 정상점수 만들기 운동에서 와슴
...,...,...,...,...,...,...,...,...
110394,2003,킹덤,TV_Show,NaN,184308.0,184308.0,6.0,킹덤 만화 팬이라면 볼 만함. 근데 주인공이 연기를 너무 못함. 입다물고 액션만 했...
110395,2003,킹덤,TV_Show,NaN,184308.0,184308.0,6.0,나름 잼나게 보긴 했는데... 화질이 안 좋아 보였어요. 마치 저화질 영상을 큰 화...
110396,2003,킹덤,TV_Show,NaN,184308.0,184308.0,4.0,일본특유에 오버액션이 아주 빛을 바랫고 하도 쁘액쁘액 질러대서 상영후 달팽이관이 7...
110397,2003,킹덤,TV_Show,NaN,184308.0,184308.0,6.0,영화는 재밌는데 대사가 뇌절... 너무 오글거려서 이쯤 그만했으면 좋겠는데 꼭 한소...


In [16]:
len(df['title'].unique())

131

In [31]:
from tqdm import tqdm

In [43]:
norm_reviews = [normalize(review) for review in df['text']]

KeyboardInterrupt: 

In [45]:
len(norm_reviews)

774

In [ ]:
df['norm_reviews'] = norm_reviews

In [32]:
title_list = list(df['title'].unique())
keywords_list = []

for title in tqdm(title_list):
    reviews = list(df[df['title']==title].text)
    norm_reviews = [normalize(review) for review in reviews]
    keywords = keyword_extraction(reviews)
    print(keywords)
    keywords_list.append(keywords)

  0%|          | 0/131 [00:00<?, ?it/s]

scan vocabs ... 
num vocabs = 1742
iter = 10

  1%|          | 1/131 [01:00<2:10:01, 60.01s/it]

done
['정만식', '연기', '배우', '오랜만에']
None
scan vocabs ... 
num vocabs = 1791
iter = 10

  2%|▏         | 2/131 [02:05<2:15:48, 63.16s/it]

...

done
['전쟁', '최고의', '명작', '말이']
None
scan vocabs ... 
num vocabs = 1340
iter = 10

 69%|██████▊   | 90/131 [1:26:17<33:58, 49.73s/it]

done
['미국', '정치', '연기', '베일']
None


 69%|██████▊   | 90/131 [1:26:31<39:25, 57.69s/it]


KeyboardInterrupt: 

In [34]:
print(keywords_list)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [35]:
keywords_list

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]